# Практическая работа: Анализ временных рядов и прогнозирование

## Цель работы
Освоить практические навыки анализа и прогнозирования временных рядов, включая декомпозицию, тестирование стационарности, построение ARIMA моделей и применение машинного обучения.

## Задания для выполнения
В этой практической работе вы выполните 8 заданий, каждое из которых направлено на освоение определенного аспекта анализа временных рядов.

**Важно:** Используйте TimeSeriesSplit для валидации моделей машинного обучения!

In [ ]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Библиотеки для временных рядов
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Машинное обучение
from sklearn.model_selection import TimeSeriesSplit, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Настройка графиков
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
sns.set_palette("husl")

# Для воспроизводимости результатов
np.random.seed(42)

print("✅ Библиотеки успешно импортированы")

## Создание датасета

Создадим синтетический временной ряд с известными компонентами для практики.

In [ ]:
# Создание синтетического временного ряда
n = 730  # 2 года дневных данных
dates = pd.date_range(start='2022-01-01', periods=n, freq='D')

# Компоненты временного ряда
time = np.arange(n)
trend = 0.03 * time + 100  # Линейный тренд
seasonal_yearly = 15 * np.sin(2 * np.pi * time / 365)  # Годовая сезонность
seasonal_weekly = 5 * np.sin(2 * np.pi * time / 7)  # Недельная сезонность
noise = np.random.normal(0, 3, n)  # Случайный шум

# Аддитивная модель
ts_values = trend + seasonal_yearly + seasonal_weekly + noise

# Создание DataFrame
df = pd.DataFrame({
    'date': dates,
    'value': ts_values
})
df.set_index('date', inplace=True)

print(f"Создан временной ряд:")
print(f"  Период: {df.index[0].date()} - {df.index[-1].date()}")
print(f"  Количество наблюдений: {len(df)}")
print(f"\nПервые 5 значений:")
print(df.head())
print(f"\nОписательная статистика:")
print(df.describe())

## TODO 1: Визуализация и первичный анализ временного ряда

**Задание:**
1. Постройте график временного ряда
2. Добавьте скользящее среднее (rolling mean) с окном 30 дней
3. Добавьте доверительный интервал (rolling mean ± 2*rolling std)
4. Проанализируйте визуально: есть ли тренд? сезонность? выбросы?
5. Постройте гистограмму распределения значений
6. Постройте boxplot по месяцам для выявления сезонности

**Подсказка:** Используйте `df.rolling(window=30)` для скользящих статистик

In [ ]:
# TODO: Ваш код здесь
# Визуализация временного ряда со скользящим средним


In [ ]:
# TODO: Ваш код здесь
# Гистограмма распределения


In [ ]:
# TODO: Ваш код здесь
# Boxplot по месяцам


## TODO 2: Декомпозиция временного ряда

**Задание:**
1. Выполните декомпозицию временного ряда (используйте `seasonal_decompose`)
2. Попробуйте аддитивную модель с периодом 365
3. Попробуйте мультипликативную модель с периодом 365
4. Визуализируйте компоненты обеих декомпозиций
5. Сравните остатки (residuals) аддитивной и мультипликативной моделей
6. Определите, какая модель лучше подходит для данных и почему
7. Проанализируйте сезонную компоненту: какой паттерн повторяется?

**Требования:**
- Создайте функцию для анализа остатков (среднее, std, тест на нормальность)
- Постройте гистограмму остатков
- Сделайте выводы в текстовой ячейке

In [ ]:
# TODO: Ваш код здесь
# Аддитивная декомпозиция


In [ ]:
# TODO: Ваш код здесь
# Мультипликативная декомпозиция


In [ ]:
# TODO: Ваш код здесь
# Функция для анализа остатков
def analyze_residuals(residuals, title='Остатки'):
    """
    Анализ остатков декомпозиции:
    - Описательная статистика
    - Тест на нормальность
    - Визуализация
    """
    # TODO: Реализуйте функцию
    pass

# Применение функции к остаткам

**TODO: Ваши выводы здесь**

Анализ декомпозиции:
- Какая модель лучше (аддитивная или мультипликативная) и почему?
- Что показывает тренд?
- Какой период сезонности наблюдается?
- Являются ли остатки белым шумом?

## TODO 3: Тестирование стационарности

**Задание:**
1. Реализуйте функцию `test_stationarity()` которая:
   - Выполняет ADF тест
   - Выполняет KPSS тест
   - Выводит результаты обоих тестов
   - Делает вывод о стационарности
   - Визуализирует ряд с rolling mean и rolling std

2. Примените функцию к исходному ряду

3. Если ряд нестационарен, примените преобразования:
   - Первое дифференцирование
   - Логарифмирование (если значения положительные)
   - Логарифмирование + дифференцирование

4. Протестируйте каждое преобразование

5. Выберите оптимальное преобразование для достижения стационарности

**Важно:** 
- ADF: H₀ = ряд нестационарен, p < 0.05 → стационарен
- KPSS: H₀ = ряд стационарен, p >= 0.05 → стационарен

In [ ]:
# TODO: Ваш код здесь
def test_stationarity(timeseries, title='Временной ряд'):
    """
    Комплексное тестирование стационарности:
    1. ADF тест
    2. KPSS тест
    3. Визуализация с rolling статистиками
    4. Итоговый вывод
    
    Параметры:
    timeseries: pd.Series - временной ряд
    title: str - название для графиков
    
    Возвращает:
    dict с результатами тестов
    """
    # TODO: Реализуйте функцию
    pass

# Тестирование исходного ряда

In [ ]:
# TODO: Ваш код здесь
# Преобразования временного ряда

# 1. Первое дифференцирование
df['diff_1'] = None  # TODO

# 2. Логарифмирование (если значения положительные)
df['log'] = None  # TODO

# 3. Логарифмирование + дифференцирование
df['log_diff'] = None  # TODO

# Тестирование каждого преобразования

**TODO: Ваши выводы здесь**

Результаты тестирования стационарности:
- Является ли исходный ряд стационарным?
- Какое преобразование делает ряд стационарным?
- Какое значение параметра d использовать в ARIMA?

## TODO 4: Анализ ACF и PACF, определение параметров ARIMA

**Задание:**
1. Постройте ACF и PACF графики для стационарного ряда
2. Проанализируйте графики:
   - Определите порядок AR (p) по PACF
   - Определите порядок MA (q) по ACF
   - Учтите параметр d из предыдущего задания
3. Создайте таблицу с несколькими вариантами параметров (p, d, q)
4. Для каждого варианта обучите ARIMA модель
5. Сравните модели по AIC и BIC
6. Выберите лучшую модель

**Подсказка:** 
- PACF обрывается после лага p → AR(p)
- ACF обрывается после лага q → MA(q)
- Обычно p, q ∈ {0, 1, 2, 3}

In [ ]:
# TODO: Ваш код здесь
# Построение ACF и PACF для стационарного ряда


In [ ]:
# TODO: Ваш код здесь
# Сравнение различных моделей ARIMA

# Разделение на train/test (80/20)
train_size = int(len(df) * 0.8)
train = df['value'][:train_size]
test = df['value'][train_size:]

print(f"Train: {len(train)}, Test: {len(test)}")

# Варианты параметров для тестирования
param_combinations = [
    # TODO: Добавьте варианты (p, d, q)
    # Например: (1, 1, 1), (2, 1, 1), (1, 1, 2), ...
]

results = []

# TODO: Для каждой комбинации параметров:
# 1. Обучите модель ARIMA
# 2. Получите AIC и BIC
# 3. Сохраните результаты

# TODO: Создайте DataFrame с результатами и отсортируйте по AIC

**TODO: Ваши выводы здесь**

Выбор параметров ARIMA:
- Какие значения p и q вы определили по графикам?
- Какая модель имеет наименьший AIC?
- Какая модель выбрана как финальная и почему?

## TODO 5: Обучение и диагностика ARIMA модели

**Задание:**
1. Обучите выбранную ARIMA модель на обучающих данных
2. Выведите summary модели и проанализируйте:
   - Коэффициенты и их значимость (p-values)
   - AIC, BIC
3. Проведите диагностику остатков:
   - Постройте график остатков
   - Гистограмму остатков
   - Q-Q plot для проверки нормальности
   - ACF остатков (должна быть близка к нулю)
   - Тест Ljung-Box для автокорреляции остатков
4. Сделайте прогноз на тестовый период
5. Визуализируйте прогноз с доверительным интервалом
6. Вычислите метрики: MAE, RMSE, MAPE

**Важно:** Хорошая модель → остатки = белый шум (нет автокорреляции)

In [ ]:
# TODO: Ваш код здесь
# Обучение финальной ARIMA модели


In [ ]:
# TODO: Ваш код здесь
# Диагностика остатков


In [ ]:
# TODO: Ваш код здесь
# Прогнозирование и визуализация


In [ ]:
# TODO: Ваш код здесь
# Вычисление метрик качества

def calculate_metrics(y_true, y_pred):
    """
    Вычисление метрик для временных рядов
    """
    # TODO: Реализуйте функцию
    # MAE, RMSE, MAPE, R²
    pass

**TODO: Ваши выводы здесь**

Анализ ARIMA модели:
- Являются ли остатки белым шумом?
- Какое качество прогноза по метрикам?
- Есть ли систематические ошибки в прогнозах?

## TODO 6: Feature Engineering для машинного обучения

**Задание:**
1. Создайте функцию `create_features()` которая генерирует:
   - **Лаговые признаки**: lag_1, lag_2, lag_7, lag_14, lag_30
   - **Скользящие статистики**: rolling_mean, rolling_std, rolling_min, rolling_max (окна: 7, 14, 30)
   - **Временные признаки**: день недели, день месяца, месяц, квартал, is_weekend
   - **Разностные признаки**: diff_1, diff_7, pct_change
   - **EMA**: exponential moving average (7, 30)

2. Примените функцию к данным

3. Обработайте пропущенные значения

4. Разделите на X (признаки) и y (целевая переменная)

5. Разделите на train/test с сохранением временного порядка (80/20)

6. Масштабируйте признаки (StandardScaler)

**Важно:** 
- НЕ перемешивайте данные!
- Используйте только .shift() для лагов (не заглядываем в будущее)
- Fit scaler только на train данных

In [ ]:
# TODO: Ваш код здесь
def create_features(data, lags=[1, 2, 7, 14, 30], rolling_windows=[7, 14, 30]):
    """
    Создание признаков для машинного обучения на временных рядах
    
    Параметры:
    data: pd.Series - временной ряд
    lags: list - лаги для создания
    rolling_windows: list - размеры окон для rolling статистик
    
    Возвращает:
    pd.DataFrame с признаками
    """
    # TODO: Реализуйте функцию
    pass

# Применение функции

In [ ]:
# TODO: Ваш код здесь
# Подготовка данных для ML

# 1. Удаление пропусков

# 2. Разделение на X и y

# 3. Train/test split (временной порядок!)

# 4. Масштабирование

## TODO 7: Обучение ML моделей с TimeSeriesSplit

**Задание:**
1. Обучите минимум 3 различные модели:
   - Ridge Regression
   - Random Forest Regressor
   - Gradient Boosting Regressor

2. Для каждой модели:
   - Используйте TimeSeriesSplit (n_splits=5) для кросс-валидации
   - Вычислите среднюю метрику CV
   - Обучите на полных train данных
   - Сделайте предсказания на test данных
   - Вычислите метрики на test: MAE, RMSE, MAPE, R²

3. Создайте таблицу сравнения моделей

4. Визуализируйте прогнозы всех моделей на одном графике

5. Проанализируйте важность признаков (для Random Forest)

**Критически важно:** Используйте TimeSeriesSplit, НЕ обычную KFold!

In [ ]:
# TODO: Ваш код здесь
# Визуализация TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)

# TODO: Создайте визуализацию разбиений

In [ ]:
# TODO: Ваш код здесь
# Обучение и сравнение ML моделей

models = {
    'Ridge': Ridge(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, 
                                                   max_depth=5, random_state=42)
}

results = {}
predictions = {}

# TODO: Для каждой модели:
# 1. Cross-validation с TimeSeriesSplit
# 2. Обучение на train
# 3. Предсказание на test
# 4. Вычисление метрик

In [ ]:
# TODO: Ваш код здесь
# Визуализация прогнозов всех моделей


In [ ]:
# TODO: Ваш код здесь
# Анализ важности признаков (Random Forest)


**TODO: Ваши выводы здесь**

Сравнение ML моделей:
- Какая модель показала лучшее качество?
- Какие признаки наиболее важны?
- Как ML модели сравниваются с ARIMA?

## TODO 8: Итоговое сравнение и выводы

**Задание:**
1. Создайте итоговую таблицу сравнения всех моделей:
   - ARIMA
   - Ridge
   - Random Forest
   - Gradient Boosting

2. Для каждой модели укажите:
   - MAE, RMSE, MAPE, R²
   - Время обучения (опционально)
   - Сложность модели (количество параметров)

3. Создайте визуализацию сравнения метрик (bar plot)

4. Постройте график остатков для лучшей модели

5. Напишите развернутые выводы:
   - Какая модель лучше для данной задачи?
   - В каких случаях стоит использовать ARIMA?
   - В каких случаях стоит использовать ML?
   - Какие признаки наиболее важны?
   - Что можно улучшить?

6. Предложите идеи для дальнейшего улучшения:
   - Дополнительные признаки
   - Ансамблирование моделей
   - Настройка гиперпараметров
   - Использование SARIMA (если есть сезонность)

In [ ]:
# TODO: Ваш код здесь
# Итоговое сравнение всех моделей


In [ ]:
# TODO: Ваш код здесь
# Визуализация сравнения метрик


In [ ]:
# TODO: Ваш код здесь
# Анализ остатков лучшей модели


## Итоговые выводы

**TODO: Напишите развернутые выводы по результатам работы**

### 1. Общие наблюдения:
- ...

### 2. Лучшая модель:
- ...

### 3. ARIMA vs ML:
- ...

### 4. Важность признаков:
- ...

### 5. Рекомендации:
- Когда использовать ARIMA: ...
- Когда использовать ML: ...
- Идеи для улучшения: ...

## Бонусное задание (опционально)

### Задание 9 (бонус): Ансамблирование моделей

Создайте ансамбль из лучших моделей:
1. Взвешенное среднее прогнозов
2. Стекинг (мета-модель)
3. Сравните с отдельными моделями

### Задание 10 (бонус): SARIMA модель

Если в данных есть сезонность:
1. Определите сезонные параметры (P, D, Q, m)
2. Обучите SARIMA модель
3. Сравните с обычной ARIMA

### Задание 11 (бонус): Интерактивная визуализация

Создайте интерактивный график с помощью Plotly:
- Переключение между моделями
- Зум и панорамирование
- Tooltips с информацией

## Критерии оценки

**Максимум: 100 баллов + 30 бонусных**

| Задание | Баллы | Критерии |
|---------|-------|----------|
| TODO 1: Визуализация | 10 | Качество графиков, полнота анализа |
| TODO 2: Декомпозиция | 15 | Корректность декомпозиции, анализ остатков |
| TODO 3: Стационарность | 15 | Правильность тестов, выбор преобразований |
| TODO 4: ACF/PACF | 10 | Корректная интерпретация, выбор параметров |
| TODO 5: ARIMA | 15 | Качество модели, диагностика остатков |
| TODO 6: Feature Engineering | 10 | Полнота признаков, корректность реализации |
| TODO 7: ML модели | 15 | Использование TimeSeriesSplit, качество моделей |
| TODO 8: Выводы | 10 | Глубина анализа, обоснованность выводов |
| **Бонусы** | +30 | Дополнительные задания |

**Дополнительные критерии:**
- Качество кода: 5 баллов (читаемость, комментарии)
- Качество визуализаций: 5 баллов (информативность, оформление)

**Важно:**
- ❌ Использование обычной CV вместо TimeSeriesSplit: -10 баллов
- ❌ Data leakage (использование будущих данных): -15 баллов
- ✅ Все ячейки должны выполняться без ошибок